# Tavily Search API

[Tavily's Search API](https://tavily.com) is a search engine built specifically for AI agents (LLMs), delivering real-time, accurate, and factual results at speed.

We can use this as a [retriever](/docs/modules/data_connection/retrievers). It will show functionality specific to this integration. After going through, it may be useful to explore [relevant use-case pages](/docs/use_cases/question_answering) to learn how to use this vectorstore as part of a larger chain.

## Setup

The integration lives in the `langchain-community` package. We also need to install the `tavily-python` package itself.

```bash
pip install -U langchain-community tavily-python
```

We also need to set our Tavily API key.

In [ ]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability

In [ ]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Usage

In [8]:
from langchain.retrievers.tavily_search_api import TavilySearchAPIRetriever

retriever = TavilySearchAPIRetriever(k=3)

retriever.invoke("what year was breath of the wild released?")

[Document(page_content='Trending topics\nTrending topics\nThe Legend of Zelda™: Breath of the Wild\nSelect a product\nThe Legend of Zelda™: Breath of the Wild\nThe Legend of Zelda™: Breath of the Wild\nThe Legend of Zelda™: Breath of the Wild and The Legend of Zelda™: Breath of the Wild Expansion Pass Bundle\nThis item will be sent to your system automatically after purchase or Nintendo Switch Game Voucher redemption. The Legend of Zelda: Breath of the Wild Expansion Pass\nMore like this\nSuper Mario Odyssey™\nThe Legend of Zelda™: Tears of the Kingdom\nMario + Rabbids® Kingdom Battle\nThe Legend of Zelda™: Link’s Awakening\nHollow Knight\nThe Legend of Zelda™: Skyward Sword HD\nStarlink: Battle for Atlas™ Digital Edition\nDRAGON QUEST BUILDERS™ 2\nDragon Quest Builders™\nWARNING: If you have epilepsy or have had seizures or other unusual reactions to flashing lights or patterns, consult a doctor before playing video games. Saddle up with a herd of horse-filled games!\nESRB rating\nSup

## Chaining

We can easily combine this retriever in to a chain.

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

Context: {context}

Question: {question}"""
)
chain = (
    RunnablePassthrough.assign(context=(lambda x: x["question"]) | retriever)
    | prompt
    | ChatOpenAI(model="gpt-4-1106-preview")
    | StrOutputParser()
)

In [13]:
chain.invoke({"question": "how many units did bretch of the wild sell in 2020"})

'As of the end of 2020, "The Legend of Zelda: Breath of the Wild" sold over 21.45 million copies worldwide.'